In [99]:
# file similarity using knn
import sklearn
import numpy as np


In [38]:
# actual data loader not working
WORD = re.compile(r'\w+')
def read_data(file1, file2):
    with open(file1, 'rb') as f1:
        data1=f1.read().decode('utf-8')
    print(data1)
    words = WORD.findall(data1)
    print(words)
    
#     with open(file2, 'rb') as f2:
#         data2=f2.read()

read_data('./49960', './51060')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 11597: invalid start byte

In [118]:
# load the docs and queries for sample data
import re, math
from collections import Counter

WORD = re.compile(r'\w+')


def text_to_vector(docs):
    w_to_vect = []
    for doc in docs:
        words = WORD.findall(doc)
        w_to_vect.append(Counter(words))
    return w_to_vect

query = ['This is a foo bar sentence .', 'lets meet at the drinks bar']
docs = ['Hello lets go the bar tonight', 'This sentence is similar to a foo bar sentence .']

#convert the queries to word frequency counts 
v_queries = text_to_vector(query)
v_docs = text_to_vector(docs)
# define the class for documents
y_docs = [1,2]

In [119]:
# calculate cosine similarity
def calculate_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())   
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [120]:
# Given a set of docs, find the nearest neighbors 
# and return the similarity array for a query
def find_neighbors(v_query, v_docs):
    similarity = []
    for v_doc in v_docs:
        similarity.append(1-calculate_similarity(v_query, v_doc))
    return np.array(similarity)

In [121]:
# find the predicted class for set of n queries 
def find_class(k, v_queries, v_docs, y_docs):
    y_pred = []
    for v_query in v_queries:
        vote_dict = dict()
        sim_docs = find_neighbors(v_query, v_docs)
        k_sim_docs = np.argpartition(sim_docs, k-1)[:k]
        for sim in k_sim_docs:
            if y_docs[sim] not in vote_dict:
                vote_dict[y_docs[sim]] = 0
            vote_dict[y_docs[sim]] += 1/sim_docs[sim]
        y_pred.append(max(vote_dict.items(), key=lambda x: x[1])[0])
    return y_pred   

# k =1 (1-NN)
# v_queries - array of word freq counter for each quries
# v_docs - array of word freq counter for each docs
# y_docs - class labels for the docs
find_class(1, v_queries, v_docs, y_docs)

[2, 1]